
Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

I have successfully completed the task of cleaning, reformatting, and splitting the data from the file "bank_marketing.csv" in accordance with the bank's specifications. The following three CSV files have been created, ready to be imported into their PostgreSQL database and to facilitate easy integration of future marketing campaign data:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [106]:
import pandas as pd
import numpy as np

df = pd.read_csv("bank_marketing.csv")

In [107]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


Client.csv

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [109]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [110]:
client = df[["client_id", "age", "job", "marital", "education", "credit_default","mortgage"]]
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [111]:
client['job'] = client['job'].str.replace('.','_')
client['job'].head()

0    housemaid
1     services
2     services
3       admin_
4     services
Name: job, dtype: object

In [112]:
client['education'] = client['education'].str.replace('.','_')
client['education'] = client['education'].replace('unknown',np.NaN)
client['education'].head()

0       basic_4y
1    high_school
2    high_school
3       basic_6y
4    high_school
Name: education, dtype: object

In [113]:
client['credit_default'] = df['credit_default'].apply(lambda val: 1 if val == "yes" else 0)
client['credit_default'] = client['credit_default'].astype('bool')

In [114]:
client['mortgage'] = df['mortgage'].apply(lambda val: 1 if val == "yes" else 0)
client['mortgage'] = client['mortgage'].astype('bool')

In [115]:
client.to_csv('client.csv', index=False)

Campaign.csv

In [116]:
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome","campaign_outcome"]]
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


In [117]:
df['year'] = 2022
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no,2022
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no,2022
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no,2022
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no,2022
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no,2022


In [118]:
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

df['month'] = df['month'].map(month_mapping)

In [119]:
campaign['last_contact_date'] = pd.to_datetime(df[['day','month','year']])
campaign['last_contact_date'] = campaign['last_contact_date'].dt.strftime('%Y-%b-%d')

In [120]:
campaign['previous_outcome'] = campaign['previous_outcome'].apply(lambda val:1 if val=="success" else 0)
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

In [121]:
campaign['campaign_outcome'] = campaign['campaign_outcome'].apply(lambda val:1 if val=="yes" else 0)
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

In [122]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-May-13
1,1,1,149,0,False,False,2022-May-19
2,2,1,226,0,False,False,2022-May-23
3,3,1,151,0,False,False,2022-May-27
4,4,1,307,0,False,False,2022-May-03


In [123]:
campaign.to_csv('campaign.csv', index=False)

In [124]:
economics = df[["client_id", "cons_price_idx","euribor_three_months"]]
economics.to_csv('economics.csv', index=False)